# Using Parent Hamiltonian Class

Here we explain how to use the **Parent Hamiltonian** class of the module *parent_hamiltonian* of the library **PH** that can be use for computing Parent Hamiltonians of an input state

In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np

In [ ]:
import sys
sys.path.append("../../")

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS(lnnize =True)

In [ ]:
from PH.parent_hamiltonian import PH

## 1. Input Ansatz

To create a Parent Hamiltonian (**PH**) with our software we need, given an input ansatz, the complete state of the ansatz. This is the amplitudes of the state of the ansatz in the computational n qubit basis. In the module *ansatzes* of the **PH** library several ansatzes are defined and the mandatory functions for simultaing and getting results usin **Atos myqlm** are provided (see notebook **01_Ansatzes.ipynb** for a explanation and use of this module). Here we use the **ansatz_qlm_01** that is the **Atos myqlm** implementation of the ansatz in the github:

https://github.com/FumiKobayashi/Parent_Hamiltonian_as_a_benchmark_problem_for_variational_quantum_eigensolvers

from the original Parent Hamiltonian Papper:

* Kobayashi, F., Mitarai, K., & Fujii, K. (2022). Parent hamiltonian as a benchmark problem for variational quantum eigensolvers. Phys. Rev. A, 105, 052415 (https://doi.org/10.1103%2Fphysreva.105.052415)

In [ ]:
from PH.ansatzes import ansatz_qlm_01, solve_ansatz

In [ ]:
n_qubits = 8
depth = 3
ansatz_01, theta = ansatz_qlm_01(nqubits=n_qubits, depth=depth)
circ = ansatz_01.to_circ()
%qatdisplay circ --svg

Here we fix the parameters of the ansatz (random selection)

In [ ]:
parameters = list(2*np.pi * np.random.rand(len(theta)))
parameters = {v_ : parameters[i] for i, v_ in enumerate(theta)}
print(parameters)

In [ ]:
pdf01 = solve_ansatz(ansatz_01, parameters, qpu_mps)

The output is a pandas DataFrame with the complete output ibfirmnation of the state of the ansatz

In [ ]:
pdf01.head()

In [ ]:
amplitudes = list(pdf01['Amplitude'])

## 2. Parent Hamiltonian Class

The main step is instantiate the **PH** python class from *parent_hamiltonian* module of **PH** library. For this we need to provided the amplitudes of the ansatz state as a python list.



In [ ]:
ph_object = PH(amplitudes)

### 2.1 Naive Parent Hamiltonian Computations

The **PH** class allows to compute the Parent Hamiltonian for an all-to-all interaction. Here the methods of attributes related with this capability will be explained.

#### Density matrix

The class allows to compute the density matrix of the state by executing the method **get_density_matrix**. The attribute *rho* stores the computed density matrix

In [ ]:
ph_object.get_density_matrix()
print(ph_object.rho.shape)

#### Projector over the null sapce

For computing the parent hamiltonian of an input matrix the method **get_parent_hamiltonian** can be used. In fact the method returns the complete sum of all the projectors onto the null space of the input matrix

In [ ]:
ph_ = ph_object.get_parent_hamiltonian(ph_object.rho)

In [ ]:
ph_.shape

The application of the result over the amplituds should be return 0

In [ ]:
np.isclose(ph_ @ ph_object.amplitudes, 0).all()

#### Decomposition in Generalized Pauli matrices basis

Given an input matrix the Pauli decomposition is provided by using the **pauli_decomposition** method. This method returns the coeficients of the pauli matrices and the pauli matrices as Pauli Strings.

**Note:** for the naive computation this decomposition can be very memory and time consuming so it is limited to 11 qubits

In [ ]:
coefs, paulis = ph_object.pauli_decomposition(ph_, n_qubits)

In [ ]:
len(coefs) == 4**n_qubits

Here each coeficient element corresponds to same element of the Pauli String

In [ ]:
print(coefs[:10])

In [ ]:
paulis[0:10]

As can be seen in the naive computation all the qubits have asignated a Pauli string, this is because the Hamiltonian obtained by the naive method is not local and assumes an all-to-all iteration between the qubits.

#### Complete naive computation

The method **naive_ph** computes all the mandatory steps to obtain the parent hamiltonian and its decomposition in pauli strings with the associated coefficients. This method populates the following attributes:

* *pauli_coeficients*: list with the coeficients of the pauli matrices
* *pauli_matrices*: list with the pauli matrices as Pauli Strings
* *naive_parent_hamiltonian*: the parent hamiltonian
* *rho*: the density matrix

In [ ]:
ph_object = PH(amplitudes)

In [ ]:
ph_object.naive_ph()

In [ ]:
ph_object.pauli_coeficients[:10]

In [ ]:
ph_object.pauli_matrices[:10]

In [ ]:
ph_object.naive_parent_hamiltonian.shape

In [ ]:
ph_object.rho.shape

### 2.2. Local Parent Hamiltonian

The class allows to compute the local Parent Hamiltonian. Here the methods of attributes related with this capability will be explained.

In [ ]:
ph_object = PH(amplitudes)

#### Reduced Density Matrix

The **get_reduced_density_matrices** method computes the minimum reduced density matrix associated at each qubit that have a null space (the kernel computation can be done). The method poupulates following attributes:

* reduced_rho: list where each element is a reduced density matrix asociated to a qubit.
* local_free_qubits: list where each elements is a list with the qubits affected by the correspondent reduced density matrix.

So for *reduced_rho[i]* the affected qubits will be *local_free_qubits[i]*

In [ ]:
ph_object.get_reduced_density_matrices()

In [ ]:
ph_object.local_free_qubits

In [ ]:
# Reduced Density Matrix for qubit j
j = 2
print("The element: {} have a reduced density matrix of shape: {} and affects to qubits: {}".format(
    j, ph_object.reduced_rho[j].shape, ph_object.local_free_qubits[j]
))

Due to the symetries of the **ansatz_qlm_01** the obtained reduced density matrix are equal for all the qubits of the ansazt. But this is not necesary true for other type of ansatzes

In [ ]:
np.isclose(ph_object.reduced_rho[0], ph_object.reduced_rho[1]).all()

Here we computed the reduced density matrix obtained for ansatz02

In [ ]:
from PH.ansatzes import ansatz_qlm_02

In [ ]:
n_qubits = 12
depth = 3
ansatz_02, theta_02 = ansatz_qlm_02(nqubits=n_qubits, depth=depth)
parameters_02 = list(np.random.rand(len(theta_02)))
parameters_02 = {v_ : parameters_02[i] for i, v_ in enumerate(theta_02)}
pdf_02 = solve_ansatz(ansatz_02, parameters_02, qpu_mps)
amplitudes02 = list(pdf_02['Amplitude'])
ph_anstaz02 = PH(amplitudes02)
ph_anstaz02.get_reduced_density_matrices()

In [ ]:
np.isclose(ph_anstaz02.reduced_rho[0], ph_anstaz02.reduced_rho[1]).all()

We can use the **get_parent_hamiltonian** for computing the asociated parent hamiltonian of any of the computed reduced density matrices

In [ ]:
ph_ = ph_object.get_parent_hamiltonian(ph_object.reduced_rho[j])

In [ ]:
ph_.shape

### 3.2 Complete local parent Hamiltonian Computation

The method **local_ph** creates the local parent Hamiltonian. This method populates:

* reduced_rho: list where each element is a reduced density matrix asociated to a qubit.
* local_free_qubits: list where each elements is a list with the qubits affected by the correspondent reduced density matrix.
* local_parent_hamiltonians: list where each element is the associated parent hamiltonian of the correspondient qubit
* *pauli_coeficients*: list with the coeficients of the pauli matrices
* *pauli_matrices*: list with the pauli matrices as Pauli Strings
* *qubits_list*: list with the qubits affected by the correspondient local parent hamiltonian.

*pauli_coeficients*, *pauli_matrices* and *qubits_list* are interrelated. The *j* element of the *pauli_coeficients* is the asociated pauli coeficient of the Pauli string located in *pauli_matrices[j]*. *qubits_list[j]* provides the list affected for the Pauli Strings

In [ ]:
ph_object = PH(amplitudes)

In [ ]:
ph_object.local_ph()

In [ ]:
j=0
ph_object.local_parent_hamiltonians[j].shape

In [ ]:
k = 150
print("For element: {} the Pauli string is: {}. Its asociated coefficient is: {} and the affects qubits are: {}".format(
    k, ph_object.pauli_matrices[k], ph_object.pauli_coeficients[k], ph_object.qubits_list[k]
))